In [2]:
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge geopy --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    sou

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

%matplotlib inline 

import json
from pandas.io.json import json_normalize
import requests
from sklearn.cluster import KMeans 
from geopy.geocoders import Nominatim
import folium
from bs4 import BeautifulSoup

### 1. Explore New York City

#### 1.1 Show neighborhoods.

In [4]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

In [5]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
newyork_features = newyork_data['features']
len(newyork_features)

306

In [6]:
col_ny=['Borough', 'Neighborhood', 'Latitude', 'Longitude']
df_newyork=pd.DataFrame(columns=col_ny)
df_newyork

,Borough,Neighborhood,Latitude,Longitude


In [7]:
for f in newyork_features:
    lon=f['geometry']['coordinates'][0]
    lat=f['geometry']['coordinates'][1]
    neighbor=f['properties']['name']
    borough=f['properties']['borough']
    df_newyork=df_newyork.append({'Borough': borough,
                       'Neighborhood': neighbor, 
                       'Latitude': lat, 
                       'Longitude': lon}, ignore_index=True)
df_newyork.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
df_manhattan=df_newyork[df_newyork['Borough']=='Manhattan'].reset_index(drop=True)
df_manhattan.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Here we notice that one neighborhood might be located in two boroughs. In this case, I will calculate the mean of two coordinates as the neighborhood's coordinate and combine boroughs. 

In [9]:
df_manhattan['Neighborhood'].nunique()

40

In [10]:
df_manhattan.to_csv('manhattan_neighborhoods.csv')

In [13]:
# show Manhattan map
add_newyork='Manhattan, NY'
geolocator=Nominatim(user_agent='newyork_explore')
location=geolocator.geocode(add_newyork)
location.address
ny_lat=location.latitude
ny_lon=location.longitude
map_manhattan=folium.Map(location=[ny_lat, ny_lon], zoom_start=11)

In [14]:
for lat, lng, neighbor in zip(df_manhattan['Latitude'], df_manhattan['Longitude'], df_manhattan['Neighborhood']):
    label = folium.Popup(neighbor, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan) 
map_manhattan